In [ ]:
# Read

In [1]:
import pandas as pd
A = pd.read_csv('noahs-customers.csv')
A.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468
1,1002,Julie Howell,185-1 Linden St,"Brooklyn, NY 11221",1956-12-03,680-537-8725,America/New_York,40.69426,-73.921670
2,1003,Christopher Ali,174-28 Baisley Blvd,"Jamaica, NY 11434",2001-09-20,315-846-6054,America/New_York,40.68902,-73.773470
3,1004,Christopher Rodriguez,102 Mount Hope Pl,"Bronx, NY 10453",1959-07-10,516-275-2292,America/New_York,40.84939,-73.909160
4,1005,Jeffrey Wilkinson,17 St Marks Pl,"Manhattan, NY 10003",1988-09-08,838-830-6960,America/New_York,40.72804,-73.987120


In [2]:
phones = A.phone.str.replace('-','')
A['fone'] = phones
phones.head()

0    3153775031
1    6805378725
2    3158466054
3    5162752292
4    8388306960
Name: phone, dtype: object

In [3]:
fullnames = A.name.str.split(expand=True)
midl = fullnames[1].str.lower()
last = fullnames[2].str.lower()
midl.head(),'👆middle names\n👇last names', last.head()

(0      alvarez
 1       howell
 2          ali
 3    rodriguez
 4    wilkinson
 Name: 1, dtype: object,
 '👆middle names\n👇last names',
 0    None
 1    None
 2    None
 3    None
 4    None
 Name: 2, dtype: object)

In [4]:
last = last.mask(last.isnull(),midl)
last.head()

0      alvarez
1       howell
2          ali
3    rodriguez
4    wilkinson
Name: 2, dtype: object

In [5]:
A['last'] = last
A.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,fone,last
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,3153775031,alvarez
1,1002,Julie Howell,185-1 Linden St,"Brooklyn, NY 11221",1956-12-03,680-537-8725,America/New_York,40.69426,-73.921670,6805378725,howell
2,1003,Christopher Ali,174-28 Baisley Blvd,"Jamaica, NY 11434",2001-09-20,315-846-6054,America/New_York,40.68902,-73.773470,3158466054,ali
3,1004,Christopher Rodriguez,102 Mount Hope Pl,"Bronx, NY 10453",1959-07-10,516-275-2292,America/New_York,40.84939,-73.909160,5162752292,rodriguez
4,1005,Jeffrey Wilkinson,17 St Marks Pl,"Manhattan, NY 10003",1988-09-08,838-830-6960,America/New_York,40.72804,-73.987120,8388306960,wilkinson


In [6]:
def last2fone (last):
    D={'abc':2,'def':3,'ghi':4,'jkl':5,'mno':6,'pqrs':7,'tuv':8,'wxyz':9}
    res = ''
    for char in last:
        for k,v in D.items():
            if char in k:
                res += str(v)
                break
    return res
A['last2fone'] = last.apply(last2fone)
A.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,fone,last,last2fone
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,3153775031,alvarez,2582739
1,1002,Julie Howell,185-1 Linden St,"Brooklyn, NY 11221",1956-12-03,680-537-8725,America/New_York,40.69426,-73.921670,6805378725,howell,469355
2,1003,Christopher Ali,174-28 Baisley Blvd,"Jamaica, NY 11434",2001-09-20,315-846-6054,America/New_York,40.68902,-73.773470,3158466054,ali,254
3,1004,Christopher Rodriguez,102 Mount Hope Pl,"Bronx, NY 10453",1959-07-10,516-275-2292,America/New_York,40.84939,-73.909160,5162752292,rodriguez,763744839
4,1005,Jeffrey Wilkinson,17 St Marks Pl,"Manhattan, NY 10003",1988-09-08,838-830-6960,America/New_York,40.72804,-73.987120,8388306960,wilkinson,945546766


In [7]:
A[A.fone == A.last2fone]

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,fone,last,last2fone
207,1208,Sam Tannenbaum,221 Banker St,"Brooklyn, NY 11222",1994-09-13,826-636-2286,America/New_York,40.7257,-73.9555,8266362286,tannenbaum,8266362286
